<a href="https://colab.research.google.com/github/leeyunche/hello-colab/blob/main/Mask_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import requests
import json

# 利用 requests 對 API 來源發送一個請求
url = "https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json"
response = requests.get(url)

# 將請求回應的內容存成一個字串格式
s = response.text

# 將長得像 json 格式的字串解析成字典或列表
data = json.loads(s)

In [6]:
# 計算各地區藥局數量
med_count = {}

# 填入欄位名稱
for s in data["features"]:
    county = s["properties"]["address"][:3]
    med_count[county] = med_count.get(county, 0) + 1  # 若county在字典中，返回縣市出現的次數再加1，若無則返回0

print(med_count)

{'臺北市': 340, '高雄市': 424, '臺中市': 436, '臺南市': 280, '基隆市': 57, '新竹市': 39, '嘉義市': 67, '新北市': 507, '桃園市': 271, '新竹縣': 48, '宜蘭縣': 76, '苗栗縣': 57, '彰化縣': 181, '南投縣': 67, '雲林縣': 131, '嘉義縣': 84, '屏東縣': 140, '澎湖縣': 11, '花蓮縣': 47, '臺東縣': 23, '金門縣': 6, '連江縣': 1}


In [7]:
# 計算出每個地區的成人剩餘口罩數量，將結果從大到小排列

mask_count = {}

# 填入欄位名稱
for s in data["features"]:
    county = s["properties"]["address"][:3]
    mask_count[county] = mask_count.get(county, 0) + s["properties"]["mask_adult"]

# 將結果從大到小排列
mask_count = dict(sorted(mask_count.items(), key=lambda item: item[1], reverse = True))

print(mask_count)

{'新北市': 677660, '臺中市': 556440, '高雄市': 428940, '臺南市': 408400, '桃園市': 345600, '臺北市': 323660, '彰化縣': 219400, '雲林縣': 199470, '屏東縣': 184250, '苗栗縣': 134180, '嘉義縣': 113180, '嘉義市': 107840, '宜蘭縣': 106900, '南投縣': 91240, '基隆市': 80100, '花蓮縣': 57330, '新竹縣': 53760, '新竹市': 45630, '臺東縣': 28510, '金門縣': 15510, '澎湖縣': 12600, '連江縣': 7930}


In [ ]:
# 資料庫操作實作

In [8]:
import sqlite3
import datetime
import requests
import json

In [10]:
# 將各地區的藥局數量存入資料庫的 pharmacies 表格，欄位包含「地區」、「數量」、「新增時間」

conn = sqlite3.connect('example.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM pharmacies''')
conn.commit()

# 新增資料
url = "https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json"
response = requests.get(url)
s = response.text
data = json.loads(s)

# 整理資料
med_count = {}
for s in data["features"]:
    county = s["properties"]["address"][0:3]
    med_count[county] = med_count.get(county, 0) + 1

# 將資料存至 database
for city, counts in med_count.items():
    t = datetime.datetime.now()
    c.execute(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
    conn.commit()

# 查詢資料
c.execute("SELECT * FROM pharmacies")
print(c.fetchall())
conn.commit()
conn.close()

[('臺北市', '340', '2024-04-17 08:26:03.165586'), ('高雄市', '424', '2024-04-17 08:26:03.175227'), ('臺中市', '436', '2024-04-17 08:26:03.182343'), ('臺南市', '280', '2024-04-17 08:26:03.189156'), ('基隆市', '57', '2024-04-17 08:26:03.196259'), ('新竹市', '39', '2024-04-17 08:26:03.203169'), ('嘉義市', '67', '2024-04-17 08:26:03.211133'), ('新北市', '507', '2024-04-17 08:26:03.218519'), ('桃園市', '271', '2024-04-17 08:26:03.226768'), ('新竹縣', '48', '2024-04-17 08:26:03.235116'), ('宜蘭縣', '76', '2024-04-17 08:26:03.242397'), ('苗栗縣', '57', '2024-04-17 08:26:03.249913'), ('彰化縣', '181', '2024-04-17 08:26:03.256736'), ('南投縣', '67', '2024-04-17 08:26:03.265716'), ('雲林縣', '131', '2024-04-17 08:26:03.272719'), ('嘉義縣', '84', '2024-04-17 08:26:03.279713'), ('屏東縣', '140', '2024-04-17 08:26:03.287064'), ('澎湖縣', '11', '2024-04-17 08:26:03.294527'), ('花蓮縣', '47', '2024-04-17 08:26:03.301777'), ('臺東縣', '23', '2024-04-17 08:26:03.309622'), ('金門縣', '6', '2024-04-17 08:26:03.317740'), ('連江縣', '1', '2024-04-17 08:26:03.325556')]


In [11]:
# 將各地區的剩餘口罩存入資料庫的 masks 表格，欄位包含「地區」、「剩餘口罩數量」、「新增時間」

conn = sqlite3.connect('example.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS masks
             (city text, mask_remain text, createdAt datetime)''')
c.execute('''DELETE FROM masks''')
conn.commit()

# 整理資料
mask_count = {}
for s in data["features"]:
    county = s["properties"]["address"][0:3]
    mask_count[county] = mask_count.get(county, 0) + s["properties"]["mask_adult"]

# 將資料存入 database
for city, counts in mask_count.items():
  t = datetime.datetime.now()
  c.execute(f"INSERT INTO masks VALUES ('{city}', {counts}, '{t}')")
  conn.commit()

# 查詢資料
c.execute("SELECT * FROM masks")
print(c.fetchall())
conn.commit()
conn.close()

[('臺北市', '323660', '2024-04-17 08:26:17.251840'), ('高雄市', '428940', '2024-04-17 08:26:17.261421'), ('臺中市', '556440', '2024-04-17 08:26:17.270273'), ('臺南市', '408400', '2024-04-17 08:26:17.279189'), ('基隆市', '80100', '2024-04-17 08:26:17.289067'), ('新竹市', '45630', '2024-04-17 08:26:17.298175'), ('嘉義市', '107840', '2024-04-17 08:26:17.306969'), ('新北市', '677660', '2024-04-17 08:26:17.317320'), ('桃園市', '345600', '2024-04-17 08:26:17.326490'), ('新竹縣', '53760', '2024-04-17 08:26:17.335204'), ('宜蘭縣', '106900', '2024-04-17 08:26:17.344125'), ('苗栗縣', '134180', '2024-04-17 08:26:17.352835'), ('彰化縣', '219400', '2024-04-17 08:26:17.361061'), ('南投縣', '91240', '2024-04-17 08:26:17.369975'), ('雲林縣', '199470', '2024-04-17 08:26:17.379116'), ('嘉義縣', '113180', '2024-04-17 08:26:17.388050'), ('屏東縣', '184250', '2024-04-17 08:26:17.397162'), ('澎湖縣', '12600', '2024-04-17 08:26:17.407018'), ('花蓮縣', '57330', '2024-04-17 08:26:17.417503'), ('臺東縣', '28510', '2024-04-17 08:26:17.428540'), ('金門縣', '15510', '2024-04-